## 第一个模型：全连接模型

_Sequential_是实现全连接网络的最好方式。但从简单的全连接网络学习内容，需要理解这几个概念：
* 层对象接受张量为参数，返回一个张量  
* 输入是张量，输出也是张量的一个框架就是一个模型，通过_Model_定义  
* 这样的模型可以像_Sequential_一样被训练

In [1]:
from keras.layers import Input,Dense
from keras.models import Model

#This returns a tensor
inputs=Input(shape=(784,))

#a layer instance is callable on a tensor,and returns a tensor(一个层实例可以在一个张量上调用，并返回一个张量)
x=Dense(64,activation='relu')(inputs)
x=Dense(64,activation='relu')(x)
predictions=Dense(10,activation='softmax')(x)

import numpy as np
import keras
data = np.random.random((100, 784))
labels = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

#This creates a model that includes
#the input layer and three Dense Layers
model=Model(inputs=inputs,outputs=predictions)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(data,labels,epochs=10)

Epoch 1/10
100/100 [==============================] - 3s 34ms/step - loss: 2.7713 - acc: 0.1000
Epoch 2/10
100/100 [==============================] - 0s 371us/step - loss: 2.2933 - acc: 0.1700
Epoch 3/10
100/100 [==============================] - 0s 255us/step - loss: 2.2248 - acc: 0.1700
Epoch 4/10
100/100 [==============================] - 0s 296us/step - loss: 2.1571 - acc: 0.1900
Epoch 5/10
100/100 [==============================] - 0s 321us/step - loss: 2.1227 - acc: 0.2400
Epoch 6/10
100/100 [==============================] - 0s 316us/step - loss: 2.0835 - acc: 0.2200
Epoch 7/10
100/100 [==============================] - 0s 331us/step - loss: 2.0390 - acc: 0.2800
Epoch 8/10
100/100 [==============================] - 0s 351us/step - loss: 2.0189 - acc: 0.3200
Epoch 9/10
100/100 [==============================] - 0s 336us/step - loss: 1.9670 - acc: 0.3000
Epoch 10/10
100/100 [==============================] - 0s 251us/step - loss: 1.9536 - acc: 0.3400


模型都是**可调用的**的，就想层一样

利用函数式模型的接口，可以很容易的重用已经训练好的模型：你可以吧模型当做层一样，通过提供一个tensor来调用，注意：当你调用一个模型时，不仅重用了它的结构，也重用了它的权重。

In [2]:
from keras.layers import Input,Dense
from keras.models import Model

x=Input(shape=(784,))
y=model(x)

这种方式可以允许我们快速创建能处理序列信号的模型，可以很快将一个图像分类的模型变为一个队视频分类的模型，只需要一行代码

In [6]:
from keras.layers import TimeDistributed

#input tensor for sequences of 20 timesteps
#each containing a 784_dimensional vector
input_sequences=Input(shape=(20,784))
#This applies our previous model to every timestep in the input  sequences
#the output of the previous model was a 10-way softmax
#so the output of the layer below will be a sequence of 20 vectors of size 10
processed_sequences=TimeDistributed(model)(input_sequences)

## 多输入和多输出模型

使用函数式模型的一个典型场景是搭建多输入、多输出的模型。  
考虑这样一个模型，我么希望预测twitter上一条新闻被转发的次数。模型的主要输入是新闻本身，也就是一个词语的序列，但我们还可以拥有额外的输入，如日期等。这个模型的损失函数由两部分构成，辅助的损失函数评估仅仅基于新闻本身做出预测的情况，主损失函数评估基于新闻和额外信息的预测的情况，即使来自主损失函数打的梯度发生弥散，来自辅助损失函数的信息也能够训练Embedding和LSTM层。在模型中早点使用主要的损失函数是对于深度网络的一个良好的正则方法。
![](https://keras-cn.readthedocs.io/en/latest/images/multi-input-multi-output-graph.png)

主要的输入是接收新闻本身，即一个整数的序列（每个整数编码了一个词），这些整数位于1到10000之间，这个序列有100个单词

In [ ]:
from keras.layers import Input,Embedding,LSTM,Dense
from keras.models import Model
# Headline input :meant to receive sequences of 100 integers,between 1 and 10000
#Note that we can name any layer by passing it a 'name' argument
main_input=Input(shape=(100,),dtype='int32',name='main_input')

#This embedding layer will encode the input sequence
x=Embedding(output_dim=512,input_dim=10000,input_length=100)(main_input)

#A LSTM will transform the vector sequence into a single vector
#containing information about the entire sequence
lstm_out=LSTM(32)(x)

#插入一个额外的损失，使得即使在很高的主损失的情况下，LSTM和Embedding层也能平滑的训练。
auxiliary_output=Dense(1,activation='sigmoid',name='aux_output')(lstm_out)

#将LSTM于额外的输入数据串联起来组成输入，送入模型
auxiliary_input=Input(shape=(5,),name='aux_input')
x=keras.layers.concatenate([lstm_out,auxiliary_input])

#We stack a deep densely-connected network on top
x=Dense(64,activation='relu')(x)
x=Dense(64,activation='relu')(x)
x=Dense(64,activation='relu')(x)

#And finally we add the main lagoistic regression layer
main_output=Dense(1,activation='sigmoid',name='main_output')(x)

#最后，定义整个2输入，2输入的模型：
model=Model(inputs=[main_input,auxiliary_input],outputs=[main_output,auxiliary_output])

#模型定义后，编译模型。给额外的损失赋0.2的权重，可以通过关键字参数loss_weight或loss来为不同的输出设置不同的损失函数或权值。这两个参数都可以用列表
#或字典。本例中，给loss传递单个损失函数，这个损失函数会被用在所有输出上。
model.compile(optimizer='rmsprop',loss='binary_crossentropy',loss_weights=[1.0,.2])

#编译完成后，传递训练数据和标签进行训练
model.fit([headline_data,additional_data],[labels,labels],epoch=50,batch_size=32)

因为输入和输出都是被命名过的，也可以通过下面的方法编译和训练模型

In [ ]:
model.compile(optimizer='rmsprop',
                           loss={'main_output':'binary_crossentropy','aux_output':'binary_crossentropy'},
                              loss_weights{'main_output':1.0,'aux_output':'0.2'})
model.fit({'main_input':headline_data,'aux_input':additional_data},
                  {'main_output':labels,'aux_output':labels},
                       epoch=50,bathc_size=32)

## 共享层

考虑微博数据，我们希望建立模型来判别两条微博是否来自同一个用户，这个需求同样可以用来判断一个用户的两条微博的相似性。  
一种实现方法是，建立一个模型，它分别将两条微博的数据映射到两个特征向量上，然后对特征向量串联并加在一个logistic回归层，输出它们来自同一个用户的概率。这种模型的训练数据是一对对的微博。  
因为这个问题是对称的，所以处理第一条微博的模型当然也能用于处理第二条微博，所以我们使用一个共享的LSTM层来进行映射。  
首先，我们将微博的数据转为（140,256）的矩阵，即每条微博有140个字符，每个单词的特征由一个256维的词向量来表示，向量的每个元素为1表示某个字符出现，为0表示不出现，这是一个one-hot编码。  
之所以是（140,256）是因为一个微博最多有140个字符，而扩展的ASC||码表编码了常见的256个字符。当然，如果考虑中文字符，那一个单词的词向量就不止256了。


In [ ]:
import keras
from keras.layers import Input,LSTM,Dense
from keras.models import Model

tweet_a=Input(shape=(140,256))
tweet_b=Input(shape=(140,256))

#若要对不同的输入共享同一层，就初始化改层一次，然后多次调用它
#This layer can take as input a matrix
#and will return a vector of size 64
shared_lstm=LSTM(64)

#when we reuse the same layer instance multiple times,the weights of the layer and also being reused
#(it is effectively **the same** layer)
encoded_a=shared_lstm(tweet_a)
encoded_b=shared_lstm(tweet_b)

#we can then concatenate the two vectors:
merged_vector=keras.layers.concatenate([encoded_a,encoded_b],axis=-1)

#and add a logistic regression on top
predictions=Dense(1,activation='sigmoid')(merged_vector)

#we define a trainable model linking the tweet inputs to the predictions
model=Model(inputs=[tweet_a,tweet_b],outputs=predictions)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics='accuracy')
model.fit([data_a,data_b],labels,epochs=10)

## 更多的例子

### inception

inception的详细结构参见google 的论文 [going deeper with convolutions](http://arxiv.org/abs/1409.4842)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input

input_img = Input(shape=(256, 256, 3))

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)

### 卷积层的残差连接

残差网络(Residual network)的详细信息参考这篇文章[Deep Residual Learning for Image Recognition](http://arxiv.org/abs/1512.03385)

In [ ]:
from keras.layers import Conv2D, Input

# input tensor for a 3-channel 256x256 image
x = Input(shape=(256, 256, 3))
# 3x3 conv with 3 output channels (same as input channels)
y = Conv2D(3, (3, 3), padding='same')(x)
# this returns x + y.
z = keras.layers.add([x, y])

### 共享视觉模型

该模型的两个输入上重用了图像处理的模型，用以判别两个mnist数字是否是相同的数字。

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model

# First, define the vision modules
digit_input = Input(shape=(27, 27, 1))
x = Conv2D(64, (3, 3))(digit_input)
x = Conv2D(64, (3, 3))(x)
x = MaxPooling2D((2, 2))(x)
out = Flatten()(x)

vision_model = Model(digit_input, out)

# Then define the tell-digits-apart model
digit_a = Input(shape=(27, 27, 1))
digit_b = Input(shape=(27, 27, 1))

# The vision model will be shared, weights and all
out_a = vision_model(digit_a)
out_b = vision_model(digit_b)

concatenated = keras.layers.concatenate([out_a, out_b])
out = Dense(1, activation='sigmoid')(concatenated)

classification_model = Model([digit_a, digit_b], out)

### 视觉问答模型

在针对一幅图片使用自然语言进行提问时，该模型能提供关于该图片的一个单词的答案  
这个模型将自然语言的问题和图片分别映射为特征向量，将二者合并后训练一个logistic回归层，从一系列可能的回答中挑选一个

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential

# First, let's define a vision model using a Sequential model.
# This model will encode an image into a vector.
vision_model = Sequential()
vision_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
vision_model.add(Conv2D(64, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
vision_model.add(Conv2D(128, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
vision_model.add(Conv2D(256, (3, 3), activation='relu'))
vision_model.add(Conv2D(256, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Flatten())

# Now let's get a tensor with the output of our vision model:
image_input = Input(shape=(224, 224, 3))
encoded_image = vision_model(image_input)

# Next, let's define a language model to encode the question into a vector.
# Each question will be at most 100 word long,
# and we will index words as integers from 1 to 9999.
question_input = Input(shape=(100,), dtype='int32')
embedded_question = Embedding(input_dim=10000, output_dim=256, input_length=100)(question_input)
encoded_question = LSTM(256)(embedded_question)

# Let's concatenate the question vector and the image vector:
merged = keras.layers.concatenate([encoded_question, encoded_image])

# And let's train a logistic regression over 1000 words on top:
output = Dense(1000, activation='softmax')(merged)

# This is our final model:
vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# The next stage would be training this model on actual data.

### 视频问答模型

在做完图片问答模型后，我么可以快速将其转为视频问答的问题。是适当的训练下，你可以为模型提供一个短视频(100帧)，然后向模型提问一个关于该视频的问题，如‘what sport is the boy palying?’->"football"

In [ ]:
from keras.layers import TimeDistributed

video_input = Input(shape=(100, 224, 224, 3))
# This is our video encoded via the previously trained vision_model (weights are reused)
encoded_frame_sequence = TimeDistributed(vision_model)(video_input)  # the output will be a sequence of vectors
encoded_video = LSTM(256)(encoded_frame_sequence)  # the output will be a vector

# This is a model-level representation of the question encoder, reusing the same weights as before:
question_encoder = Model(inputs=question_input, outputs=encoded_question)

# Let's use it to encode the question:
video_question_input = Input(shape=(100,), dtype='int32')
encoded_video_question = question_encoder(video_question_input)

# And this is our video question answering model:
merged = keras.layers.concatenate([encoded_video, encoded_video_question])
output = Dense(1000, activation='softmax')(merged)
video_qa_model = Model(inputs=[video_input, video_question_input], outputs=output)
